# Project Phase 3: Video Moment Detection

## Frames extraction

In [3]:
import av
import av.datasets
import os

output_folder = "./frames_part3"
video = "./videos/6gyD-Mte2ZM.mp4"
# File name without extension
filename = os.path.splitext(video)[0]
if not os.path.isdir(output_folder + "/" + filename):
    os.makedirs(output_folder + "/" + filename)
with av.open(video) as container:
    stream = container.streams.video[0]
    # We want 2 frame per second
    fps = stream.average_rate
    interval = int(round(fps*0.5))
    #print(fps)
    cpt = 0
    for i,frame in enumerate(container.decode(stream)):
        if i % interval == 0:
            frame.to_image().save(output_folder+"/"+filename+"/"+str(cpt*(0.5))+".jpg", quality=80)
            cpt += 1

## Captions generations

## Captions encoding

## Regroupment based on clustering

## Frames fusionning